<img src="data/images/lecture-notebook-header.png" />

# Preparation: Data Collection Using APIs

An API (Application Programming Interface) is a set of rules and protocols that allows different software applications to communicate and interact with each other. It defines a collection of methods, data formats, and rules for how software components should interact. To collect online data using an API, you typically follow these steps:

* **Find a suitable API:** Identify an API that provides the data you need. Many websites and online services offer APIs that allow developers to access and retrieve their data programmatically. Examples include social media APIs (Twitter, Facebook), weather APIs, financial APIs, and more.

* **Read the API documentation:** Understand the API's documentation, which provides details about the available endpoints, authentication requirements, request/response formats, rate limits, and any specific guidelines or restrictions.

* **Obtain an API key or credentials:** Some APIs require authentication to access their data. This often involves obtaining an API key or credentials, which are unique identifiers that authorize your application to make requests to the API.

* **Make API requests:** Using a programming language or a tool like cURL or Postman, you can send HTTP requests to the API's endpoints. These requests typically include parameters to specify the data you want to retrieve (e.g., filters, search terms, date ranges). The API responds with the requested data in a specified format (e.g., JSON, XML).

* **Process and store the data:** Once you receive the data from the API, you can process it and extract the relevant information. This may involve parsing the response, transforming the data into a suitable format, and storing it in a database or file for further analysis or use.

* **Handle pagination and rate limits:** Depending on the API, you may need to handle pagination if the data is returned in chunks or pages. Additionally, APIs often impose rate limits to control the number of requests you can make within a given time period. It's essential to respect these limits to avoid being blocked or throttled by the API provider.

By leveraging APIs, you can access and collect online data in a structured and programmatic manner. APIs offer a standardized way to retrieve data from various online sources, allowing developers to automate data collection processes, integrate external data into their applications, and conduct analysis or research based on up-to-date information.

## Setting up the Notebook

### Import Required Packages

In [ ]:
import requests
import json

from urllib.parse import urlencode

---

## Nominatim API

As an example API for this notebook we look at Nominatim. The Nominatim API is a geocoding and reverse geocoding API provided by OpenStreetMap (OSM), a collaborative mapping project. Geocoding is the process of converting addresses or place names into geographic coordinates (latitude and longitude), while reverse geocoding involves converting coordinates into meaningful addresses or place names.

The Nominatim API allows developers to access OSM's geocoding and reverse geocoding capabilities programmatically. By sending HTTP requests to the API endpoints, developers can perform the following tasks:

* **Geocoding:** Developers can provide an address or place name as input, and the Nominatim API will return the corresponding geographic coordinates (latitude and longitude). This is useful for mapping applications, location-based services, and various geospatial analysis tasks.

* **Reverse Geocoding:** Developers can provide coordinates (latitude and longitude) as input, and the Nominatim API will return the corresponding address or place name. This is helpful when you have geographic coordinates and want to identify the associated location information.

The Nominatim API provides various parameters that allow for fine-tuning geocoding and reverse geocoding requests, such as language preference, bounding box filtering, and result formats (JSON, XML, etc.). It also supports features like handling multiple results, handling different address components (street, city, country, etc.), and providing additional details like postal codes and administrative boundaries.

It's important to note that Nominatim is a widely used API, and OpenStreetMap imposes certain usage policies to prevent abuse and ensure fair usage. Developers are generally required to include attribution to OpenStreetMap when using the Nominatim API, and they should review the specific usage policy and terms of service provided by OpenStreetMap for proper implementation and compliance.

### Nominatim Search API

The [Nominatim Search API](https://nominatim.org/release-docs/develop/api/Search/) allows you to look up a location from a textual description or address. Nominatim supports structured and free-form search queries. A common use case is to find the geolocation in terms of a latitude-longitude pair for a given address. We use this use case as an example.

The important part of any request to an API is the endpoint, i.e., the URL. In case of the Nominatim Search API -- as it can be found in the documentation linked above -- is:

In [ ]:
NOMINATIM_SEARCH_URL = 'https://nominatim.openstreetmap.org/search?{params}'

The API document also describes how to use the API w.r.t. which are valid request parameters. While the endpoint above supports around 20 parameters, we consider only the following 3:

* `q=<query>`: Free-form query string to search for. Free-form queries are processed first left-to-right and then right-to-left if that fails.
* `format=[xml|json|jsonv2|geojson|geocodejson]`: The output format of the result; we consider only JSON for our example
* `limit=<integer>`: Limit on the number of returned results. (Default: 10, Maximum: 50)

So the final URL to be send to find the geolocation of, say, `'computing drive 15'` would look as follows:

`https://nominatim.openstreetmap.org/search?q=computing drive 15&format=json&limit=1`

Of course, this URL needs to be properly encoded as characters such as spaces are not allowed in spaces. This encoding can be automatically done using different ready-made Python packages.

With this knowledge, we can write a small method that searches and returns the results for a given query

In [ ]:
def nominatim_search(query, result_format='json', limit=1):
    try:
        
        params = {
            'q': query,
            'format': result_format,
            'limit': limit
        }
        
        # Convert dictionary to query string
        query_string= urlencode(params)
        
        # Send API request
        r = requests.get(NOMINATIM_SEARCH_URL.format(params=query_string))

        # Return result list in specified format
        if result_format.lower() == 'json':
            return r.json()
        else:
            print('Error: unsupported result format (allowed: json)')
            return None
        
    except Exception as e:
        print(e)
        return None

Note that this method will always return a list of results, even if there is only 1 result.

Let's make an example request using the query from above...

In [ ]:
result_list = nominatim_search('computing drive 15')
#result_list = nominatim_search('somerset')
#result_list = nominatim_search('jcube singapore')

...and print the raw result

In [ ]:
for result in result_list:
    print(json.dumps(result, indent=2))

Firstly, the search found arguably the correct result. Of course, this is never guaranteed. The result can be empty if no matching entry is found, or it can be "wrong" if the query is too ambiguous. For example, query for "somerset", and you won't get anything related to Singapore as the top result. This is how "dirty" records and outliers can happen :).

The raw output also shows all the available details for each search result. For example, if we are only interested in the latitude and longitude of the top result, we can simply do:

In [ ]:
lat = result_list[0]['lat']
lon = result_list[0]['lon']

print('The gelocation of the top result is at lat={} and lon={}'.format(lat, lon))

Let's try another example with a more ambiguous query and check the top-10 results.

In [ ]:
result_list = nominatim_search('marina bay', limit=10)
#result_list = nominatim_search('woodlands', limit=10)
#result_list = nominatim_search('st. andrew\'s cathedral', limit=10)

for result in result_list:
    name, importance = result['display_name'], result['importance']
    print(name, '/', importance)

You can see just using these examples, that API requests based on search queries or similar potentially vague parameters can be tricky. On the one hand, you should try to to ensure unambiguous queries (in practice often impossible) and to some extent, you're always at the mercy of the API providers ranking in case of multiple matches.

### Nominatim Reverse API

The [Nominatim Reverse API](https://nominatim.org/release-docs/develop/api/Reverse/) works by finding the closest suitable OSM object and returning its address information. Again, we first need the endpoint URL which we can find in the documentation:

In [ ]:
NOMINATIM_REVERSE_URL = 'https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&{params}'

In this case, the parameters `lat` and `lon` are mandatory. Again, the endpoint accepts a series if additional parameters, but we focus on just 2 of them:

* `zoom=[0-18]`: Level of detail required for the address. Default: 18 (building level)
* `format=[xml|json|jsonv2|geojson|geocodejson]`: The output format of the result; we consider only JSON for our example

Analogous to above, we can write a method that sends a valid request to this API endpoint.

In [ ]:
def nominatim_reverse(lat, lon, result_format='json', zoom=18):
    try:
        
        params = {
            'format': result_format,
            'zoom': zoom
        }
        
        # Convert dictionary to query string
        query_string= urlencode(params)
        
        # Send API request
        r = requests.get(NOMINATIM_REVERSE_URL.format(lat=lat, lon=lon, params=query_string))

        # Return result list in specified format
        if result_format.lower() == 'json':
            return r.json()
        else:
            print('Error: unsupported result format (allowed: json)')
            return None
        
    except Exception as e:
        print(e)
        return None

Let's send an example request. The used latitude and longitude are a location inside the COM2 building

In [ ]:
nominatim_reverse(1.2942645023251518, 103.7740238293016) # COM 2 building

Compared to the Search API, the Reverse API returns only one result, as described in the documentation. In this example, we can see that the result is as expected.

We can also send the request with different values for the `zoom` parameter. Valid values are 3, 5, 8, 10, 14, 16, 17 and 18. Again, this is all documented, of course.

In [ ]:
for zoom in [3, 5, 8, 10, 14, 16, 17, 18]:
    result = nominatim_reverse(1.2942645023251518, 103.7740238293016, zoom=zoom)
    name = result['display_name']
    print('[Zoom={}] {}'.format(zoom, name))

Which `zoom` level to choose typically depends on the application context. For any kind of location-based services such as tourist navigation, you aim for maximum granularity. If you monitor natural phenomena (e.g., earthquakes or lightning strikes) or weather patterns, such a level of detail is usually not required.

---

## Summary

Collecting data using APIs is in principle very straightforward. The whole purpose of an API is to provide a well-defined interface to access the data (or to submit data). As such, basically APIs are properly documented telling you how to make a request (e.g., which parameters to set and how), and how to interpret the result. This holds true across all APIs. However, the Nominatim API is particularly convenient as it is public and unrestricted (as far as I'm aware). For many other APIs, you have to consider:

* **Access Credentials:** Most popular APIs restrict their access to registered accounts. While the registration and the access credential is often for free, there might be some hurdles. For example, in the case of the Twitter API, you have to describe what your intention of using the API is.
* **Rate Limit:** Most popular APIs have rate limits, that means that you can make only a certain number of requests within a specified time interval. If you reach the limit, depending on the API provider, 2 things can happen. In the "harmless" case, your request get rejected with an error. However, some APIs such as Google might start billing you or exceed certain rates (Google requires you to enter credit card details before providing access credentials).

But again, this is all very well documented.

Lastly, for many popular APIs there are Python packages available to simply make requests. What those packages basically do is to implement methods like `nominatim_search()` and `nominatim_reverse()` for you to use. So all the details of making the HTTP requests are hidden from you. For example, [GeoPy](https://geopy.readthedocs.io/en/stable/) is a popular package that provides methods to access a multitude of geolocation APIs including Nominatim. It's generally recommended to use such packages. Firstly, they make life easier, and secondly, they are typically mature and tested.